In [ ]:
#!pip install update pyvisa

'pip' is not recognized as an internal or external command,
operable program or batch file.


INITIALIZE

In [4]:
import pyvisa
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import csv
%matplotlib qt

In [5]:
def update_plot(fig, ax, line, lstVoltage, lstAbsCurrent):
    line.set_data(lstVoltage, lstAbsCurrent)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()
    fig.canvas.flush_events()

In [6]:

rm = pyvisa.ResourceManager()

### Keitley 2400

In [7]:
# rm = pyvisa.ResourceManager()




# Keithley 2400
# keith1 = rm.open_resource('GPIB0::24::INSTR')
# keith1.timeout = 100000
# keith1.write('*RST')
# Keithley 2400
keith = rm.open_resource('GPIB0::23::INSTR')
keith.timeout = 100000
# keith.write('*RST')
  



VisaIOError: VI_ERROR_BERR (-1073807304): Bus error occurred during transfer.

In [ ]:
outputSettings = [0.02, 10]
durationSettings = [100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100, \
                    100, 100,  
                    100, 100 ] # odd values turn ON, even values turn OF in s
def setupKL(output_settings):
    current, vCompliance = output_settings
    keith.write('*RST')
    keith.write(':ROUT:TERM FRON')
    keith.write(':SOUR:FUNC CURR')
    vCompliaceStr = ':SENSE:CURR:PROT '+ str(vCompliance)
    keith.write(':SENSE:VOLT:PROT:LEVEL 20')
    keith.write(f':SOUR:CURR:RANG {current:0.02f}')
def turnONKL(output_settings, duration_settings):
    onDurations = durationSettings[0::2]
    offDurations = durationSettings[1::2]
    #setupKL(output_settings)
    
    for i in range(len(onDurations)):
        # keith.write(":SOUR:CURR:STAR %0.2f" % Vstart)
        # keith.write(":SOUR:CURR:STOP %0.2f" % Vstop)
        # keith.write(":SOUR:CURR:STEP 0" % Vstep)
        keith.write(':OUTP ON')
        time.sleep(onDurations[i])
        keith.write(':OUTP OFF')
        time.sleep(offDurations[i])
turnONKL(outputSettings, durationSettings)

        
        


VisaIOError: VI_ERROR_BERR (-1073807304): Bus error occurred during transfer.

In [ ]:
def fncMeasureOneKL(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 1
    iNPLC = 1
    delay = 0.1
    keith.write('*RST')
    keith.write(':ROUT:TERM REAR')
    keith.write(':SOUR:FUNC VOLT')
    keith.write(':SENS:FUNC "CURR:DC"')
    keith.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    keith.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    keith.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange = ':SOUR:SWE:RANG ' + vRange
    keith.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    keith.write(iCompliance)
    keith.write(":SOUR:VOLT:MODE SWE")
    keith.write(":SENSE:CURR:PROT 1" )
    keith.write(':SENSE:CURR:NPLC %f' %iNPLC )
    keith.write(':SOUR:SWE:SPAC LIN')
    keith.write("TRAC:CLE")
    keith.write('TRAC:POIN %f' %stepNum)
    keith.write(':FORM:ELEM:SENS VOLT,CURR')
    keith.write(':TRIG:COUN %f' % stepNum)
    keith.write(':SOUR:SWE:POIN %f' %stepNum)
    keith.write(':SOUR:DEL %0.2f' % delay)
    keith.write(':OUTP ON')
    keith.write(":INIT")
    keith.write('*WAI')
    keith.write(':OUTP OFF')

    time.sleep(1)

    data = keith.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2400 and write to the file
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    #plt.xticks(fontsize = 18)
    ax.set_ylabel('Current (A)')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')


In [ ]:
def fncMeasureOneKL1(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 1
    iNPLC = 1
    delay = 0.1
    keith1.write('*RST')
    keith1.write(':ROUT:TERM REAR')
    keith1.write(':SOUR:FUNC VOLT')
    keith1.write(':SENS:FUNC "CURR:DC"')
    keith1.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    keith1.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    keith1.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange1 = ':SOUR:SWE:RANG ' + vRange
    keith1.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    keith1.write(iCompliance)
    keith1.write(":SOUR:VOLT:MODE SWE")
    keith1.write(":SENSE:CURR:PROT 1" )
    keith1.write(':SENSE:CURR:NPLC %f' %iNPLC )
    keith1.write(':SOUR:SWE:SPAC LIN')
    keith1.write("TRAC:CLE")
    keith1.write('TRAC:POIN %f' %stepNum)
    keith1.write(':FORM:ELEM:SENS VOLT,CURR')
    keith1.write(':TRIG:COUN %f' % stepNum)
    keith1.write(':SOUR:SWE:POIN %f' %stepNum)
    keith1.write(':SOUR:DEL %0.2f' % delay)
    keith1.write(':OUTP ON')
    keith1.write(":INIT")
    keith1.write('*WAI')
    keith1.write(':OUTP OFF')

    time.sleep(1)

    data = keith1.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2400 and write to the file
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    #plt.xticks(fontsize = 18)
    ax.set_ylabel('Current (A)')
    ax.set_yscale('log')
    #plt.yticks(fontsize = 18)
    # Function to update plot
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')


In [ ]:
def fncMeasureOne(arrVMeasurement, \
                  device= 'pico',\
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False):
    if (device == 'pico'):
        fncMeasureOnePico(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith'):
        fncMeasureOneKL(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'keith1'):
        fncMeasureOneKL1(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2657A'):
        fncMeasureOneM2657A(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2461SWETSP'): #TSP means using TSP commands, SWE means sweep the whole thing from start to stop in one run
        fncMeasureOneM2461SWETSP(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    elif (device == 'M2461POITSP'): #POI means set each point and then measure and move to another.
        fncMeasureOneM2461POITSP(arrVMeasurement, Sample, sweepNum, folderPath, askToStart)
    else:
        print("Incorrect device name")

In [ ]:
def fncMeasureMultiple(settings =['pico', [[0, 0.5, 0.1]]], \
                  Sample = 'Test',\
                  startNum = 1, \
                  folderPath='230606\\Dev02\\Test',\
                  askToStart=False): 
    sweepNum = 0
    dev, arrVMeasurements = settings
    for j in range (len(arrVMeasurements)):
        sweepNum = j + startNum
        fncMeasureOne(arrVMeasurements[j], dev , Sample, sweepNum, folderPath)
        time.sleep(5)

In [ ]:
folderPath = '250327_Fab241201_IrOxNewSM'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
    
folderPath = folderPath+ '\\Dev08'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")

folderPath = folderPath+ '\\H02'
# Create a new directory
try:
    os.mkdir(folderPath)
except FileExistsError:
    print("Folder already exists")
sample = folderPath[-3:]

Folder already exists
Folder already exists


In [ ]:

m2461 = rm.open_resource('TCPIP0::132.181.53.57::inst0::INSTR')
m2461.timeout = 10000
m2461.write(f"reset()")

9

In [2]:
rm.close()

NameError: name 'rm' is not defined

In [9]:
def setM2461IVMeasurementTSP (terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "2WIRE",\
                  iNPLC = 1,\
                  delay = 0.1): #(iRange = "AUTO", iLimit = 7, nplc = 1, delay = 0.1)
    # Reset and initialize instrutment
    m2461.write(f"reset()")
    m2461.write(f"status.reset()")
    m2461.write(f"errorqueue.clear()")

    m2461.write(f"reset()")
    m2461.write(f"defbuffer1.clear()")
    m2461.write(f"smu.terminals = smu.TERMINALS_{terminal}" )
    m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
    if vRange == "AUTO":
        m2461.write(f"smu.source.autorange = smu.ON")  
    else:
        m2461.write(f"smu.source.autorange = smu.OFF")
        m2461.write(f"smu.source.autorange = {vRange}") 
    
    #Set up current compliance
    m2461.write(f"smu.source.ilimit.level = %f" % iCompliance)

    # Setup current measurement mode
    m2461.write(f"smu.measure.sense = smu.SENSE_{iMeasMode}")

    #Set up current measurement range
    if iRange == "AUTO":
        m2461.write(f"smu.measure.autorange = smu.ON")
    else:
        m2461.write(f"smu.measure.autorange = smu.OFF")
        m2461.write(f"smu.measure.rangei = %f" % iRange)

    #Set the measurement integration time and delay
    m2461.write(f"smu.measure.nplc = %f" % iNPLC)
    m2461.write(f"smu.source.delay = %f" % delay)

    m2461.write(f"beeper.beep(0.5,100)")
    return True
setM2461IVMeasurementTSP()   
"""  
    
    #m2461.write(f"display.changescreen(display.SCREEN_HOME)")

    
def measureM2461OnePoint(setV, outputON = False, resetV = True):
    #setM2461IVMeasurement()
    m2461.write(f"smu.source.levelv = %f" %setV)
    
    #Configure the reading buffers
    m2461.write(f"dataBuffer = buffer.make(1)")
    m2461.write(f"dataBuffer.clear()")
    m2461.write(f"smu.nvbuffer1.appendmode = 1")
    m2461.write(f"smu.nvbuffer1.collecttimestamps = 1")
    m2461.write(f"smu.nvbuffer1.collectsourcevalues = 0")
    m2461.write(f"smu.nvbuffer1.fillmode = smua.FILL_ONCE")
    m2461.write(f"smu.nvbuffer2.clear()")
    m2461.write(f"smu.nvbuffer2.appendmode = 1")
    m2461.write(f"smu.nvbuffer2.collecttimestamps = 1")
    m2461.write(f"smu.nvbuffer2.collectsourcevalues = 0")
    m2461.write(f"smu.nvbuffer2.fillmode = smua.FILL_ONCE")
    
    
    if (not outputON):
        m2461.write(f"smua.source.output = 1")
    m2461.write(f"smua.measure.i(smua.nvbuffer1)")
    m2461.write(f"smua.measure.v(smua.nvbuffer2)")
    #time.sleep(2)

    #Turn off the output
    if resetV:
        m2461.write(f"smua.source.levelv = 0")
        m2461.write(f"smua.source.output = 0")
    
    m2461.write(f"printbuffer(1, 1, smua.nvbuffer1.readings, smua.nvbuffer2.readings)")
    rawData = m2461.read()
    # print("%g\t%g\t%g", smua.nvbuffer1.timestamps[1], smua.nvbuffer1.readings[1], smua.nvbuffer2.readings[1])
    arrRawData = rawData.split(",")
    #print ("rawData: ", rawData)

    return [float(arrRawData[i]) for i in range(len(arrRawData))]
setM2461IVMeasurement(0.02, 0.02, 1, 0.05)
# rur = measureM2461OnePoint(1,False,True)
# print (results)
current, voltage = measureM2461OnePoint(1,False,True)
print ("Current: ", current, "Voltage: ", voltage)"
"""

'  \n    \n    #m2461.write(f"display.changescreen(display.SCREEN_HOME)")\n\n    \ndef measureM2461OnePoint(setV, outputON = False, resetV = True):\n    #setM2461IVMeasurement()\n    m2461.write(f"smu.source.levelv = %f" %setV)\n    \n    #Configure the reading buffers\n    m2461.write(f"dataBuffer = buffer.make(1)")\n    m2461.write(f"dataBuffer.clear()")\n    m2461.write(f"smu.nvbuffer1.appendmode = 1")\n    m2461.write(f"smu.nvbuffer1.collecttimestamps = 1")\n    m2461.write(f"smu.nvbuffer1.collectsourcevalues = 0")\n    m2461.write(f"smu.nvbuffer1.fillmode = smua.FILL_ONCE")\n    m2461.write(f"smu.nvbuffer2.clear()")\n    m2461.write(f"smu.nvbuffer2.appendmode = 1")\n    m2461.write(f"smu.nvbuffer2.collecttimestamps = 1")\n    m2461.write(f"smu.nvbuffer2.collectsourcevalues = 0")\n    m2461.write(f"smu.nvbuffer2.fillmode = smua.FILL_ONCE")\n    \n    \n    if (not outputON):\n        m2461.write(f"smua.source.output = 1")\n    m2461.write(f"smua.measure.i(smua.nvbuffer1)")\n    m24

In [ ]:
def fncMeasureOneM2461POITSP(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250319\\Dev02\\Test',\
                  askToStart=False,\
                  terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "2WIRE", \
                  iNPLC = 1, \
                  delay = 0.1,\
                  ):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1

    setM2461IVMeasurementTSP(terminal, vRange, iRange, iCompliance, iMeasMode, iNPLC, delay)
    absVstart = abs(Vstart)
    absVstop = abs(Vstop)
    # Set up source range
    if abs(Vstart) > abs(Vstop): 
        m2461.write(f"smu.source.rangev = %f" % absVstart)
    else:
        m2461.write(f"smu.source.rangev = %f" % absVstop)
    
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    

    plt.show()
    time.sleep(1)
    m2461.write(f"smu.source.output = 1")
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
        
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        current, voltage = measureM2461OnePointTSP(Vstart + i*Vstep,True,False)
        
        # results = pico.query(':MEASure:CURRent?')
        # #print(results)

        # current = results.split(',')[0][:-1]
        #voltage = l
        lstVoltage.append(voltage)
        lstCurrent.append(current)
        lstAbsCurrent.append(abs(current))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs((current)) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(current) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        time.sleep(0.1)
    #Turn off the output
    m2461.write(f"smua.source.levelv = 0")
    m2461.write(f"smua.source.output = 0")

    
    dataToWrite = [['vRange: ', vRange]]
    dataToWrite.append(['iLimit: ', iLimit])
    dataToWrite.append(['iRange: ', iRange[16:]])
    dataToWrite.append(['nplc: ', nplc, 'delay: ', delay])
    dataToWrite.append(['data Number:', stepNum])
    dataToWrite.append(['V_meas', 'I','abs_I'])
    # lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    # plt.savefig(os.path.splitext(filePath)[0] + '.png')
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')

In [ ]:
m2461.write(f"beeper.beep(1,100)")
m2461.write(f"reset()")
m2461.write(f"status.reset()")
m2461.write(f"errorqueue.clear()")
# Configure source function as 2W DCVOLTS
m2461.write(f"smu.source.func = smu.FUNC_DC_VOLTAGE")
m2461.write(f"smu.measure.sense = smu.SENSE_2WIRE")
m2461.write(f"display.changescreen(display.SCREEN_HOME)")

43

In [26]:
iLimit = 7
#m2461.write(f"smu.source.ilimit = %f " % iLimit)
m2461.write(f"smu.source.ilimit.level = %f " % iLimit)

37

In [13]:
vRange = "smu.ON"
m2461.write(f"smu.source.autorange = {vRange}") 

31

In [10]:
def fncMeasureOneM2461SWETSP(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250318\\Dev02\\Test',\
                  askToStart=False,\
                  terminal = "FRONT",\
                  vRange = "AUTO",\
                  iRange = "AUTO",\
                  iCompliance = 7,\
                  iMeasMode = "2WIRE",\
                  iNPLC = 1,\
                  delay = 0.1):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    
    setM2461IVMeasurementTSP(terminal, vRange, iRange, iCompliance, iMeasMode, iNPLC, delay)
    m2461.write(f'smu.source.sweeplinear("VLinSweep", {Vstart:0.2f}, {Vstop:0.2f}, {stepNum:0.2f}, {delay:0.2f})')
    m2461.write(f'trigger.model.initiate()')
    m2461.write(f'waitcomplete()')
    m2461.write(f'printbuffer(1, {stepNum:f}, defbuffer1.sourcevalues, defbuffer1.readings)')
    m2461.write(f'waitcomplete()')
    time.sleep(0.05*stepNum)
    data = m2461.read().split(', ')
    print('Measured data: ', data)
    lstCurrent = [float(i) for i in data[1::2]] # get all odd elements, which are currents

    lstVoltage = [float(i) for i in data[0::2]] # get all even elements, which are voltages
    print (f'len current: {len(lstCurrent)}')
    print (f'len current: {len(lstVoltage)}')
    if Vstart > Vstop:
        Vstep = -abs(Vstep)
    else:
        Vstep = abs(Vstep)
    

    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A)')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    """
    m2461.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    m2461.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    m2461.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange = ':SOUR:SWE:RANG ' + vRange
    m2461.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    m2461.write(iCompliance)
    m2461.write(":SOUR:VOLT:MODE SWE")
    m2461.write(":SENSE:CURR:PROT 1" )
    m2461.write(':SENSE:CURR:NPLC %f' %iNPLC )
    m2461.write(':SOUR:SWE:SPAC LIN')
    m2461.write("TRAC:CLE")
    m2461.write('TRAC:POIN %f' %stepNum)
    m2461.write(':FORM:ELEM:SENS VOLT,CURR')
    m2461.write(':TRIG:COUN %f' % stepNum)
    m2461.write(':SOUR:SWE:POIN %f' %stepNum)
    m2461.write(':SOUR:DEL %0.2f' % delay)
    m2461.write(':OUTP ON')
    m2461.write(":INIT")
    m2461.write('*WAI')
    m2461.write(':OUTP OFF')

    time.sleep(1)

    data = m2461.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2461 and write to the file""
    """
    
    #lstCurrent = [float(e) for e in data[1::2]] # get all odd elements, which are currents

    #lstVoltage = [float(e) for e in data[0::2]] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    #print(lstCurrent)
    """
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A)')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')"
    """
    
    
    
    
    

In [29]:
#run Keithley 2400 
arrKLVMeasMulti =["M2461SWETSP",[#[0, 3, 0.01],\
                            #[0, 3, 0.01],\
                            [0, -3, -0.01]
                            ]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 12, folderPath)

250327_Fab241201_IrOxNewSM\Dev08\H02\Sweep12H02Star0_StopM3_StepM0.01.csv
Measured data:  ['7.5755451689e-07', '-4.5699694029e-12', '-0.0099987220019', '-1.6619448873e-11', '-0.020002216101', '-8.5864579336e-12', '-0.030001228675', '-1.4898061951e-11', '-0.040001746267', '-2.4652439812e-11', '-0.050001557916', '-3.2111598103e-11', '-0.060002434999', '-2.6373812856e-11', '-0.070003069937', '-2.1209749235e-11', '-0.080004014075', '-1.0881580359e-11', '-0.090002678335', '-3.0390238936e-11', '-0.10000319034', '-1.9488348435e-11', '-0.11000346392', '-2.2357261875e-11', '-0.12000310421', '-2.465245369e-11', '-0.13000175357', '-1.2029162388e-11', '-0.14000125229', '-8.5864718113e-12', '-0.15000075102', '-1.0881621992e-11', '-0.15999987721', '-1.3176758296e-11', '-0.17000065744', '-4.5700110363e-12', '-0.17999976873', '-2.1783526372e-11', '-0.1899972111', '-1.7193281521e-11', '-0.19999569654', '-4.0144616797e-11', '-0.20999477804', '-1.3750535433e-11', '-0.22001576424', '-3.0964085462e-11', '-

In [ ]:
def fncMeasureOneM2461SWE(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='250318\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = "AUTO"
    iCompliance = 7
    iNPLC = 1
    delay = 0.1
    # data = m2461.query_ascii_values('FETC?')
    # print('this is the data 1', data)
    m2461.write('*RST')
    m2461.write("TRAC:CLE")
    m2461.write(':ROUT:TERM FRON')
    m2461.write(':SOUR:FUNC VOLT')
    # vRange = ':SOUR:VOLT:RANG:' + vRange
    # print(vRange)
    m2461.write(':SOUR:VOLT:RANG:AUTO ON')
    m2461.write('SOUR:VOLT:ILIM %0.2f' % iCompliance)
    m2461.write(':SENS:FUNC "CURR"')
    m2461.write('SENS:CURR:RANG:AUTO ON')
    m2461.write(':SENSE:CURR:NPLC %f' % iNPLC )
    m2461.write('SOUR:SWE:VOLT:LIN %0.2f, %0.2f, %f, %0.2f' % (Vstart, Vstop, stepNum, delay))
    m2461.write('INIT')
    m2461.write('*WAI')
    m2461.write('TRAC:DATA? 1, %f, "defbuffer1", SOUR, READ' % stepNum)
    m2461.write('*WAI')
    data = m2461.read()
    
    #data = m2461.query_ascii_values('FETC?')
    
    print('Measured data: ', data)
    """
    m2461.write(":SOUR:VOLT:STAR %0.2f" % Vstart)
    m2461.write(":SOUR:VOLT:STOP %0.2f" % Vstop)
    m2461.write(":SOUR:VOLT:STEP %0.3f" % Vstep)
    vRange = ':SOUR:SWE:RANG ' + vRange
    m2461.write(vRange)
    iCompliance = ':SENSE:CURR:PROT '+ str(iCompliance)
    m2461.write(iCompliance)
    m2461.write(":SOUR:VOLT:MODE SWE")
    m2461.write(":SENSE:CURR:PROT 1" )
    m2461.write(':SENSE:CURR:NPLC %f' %iNPLC )
    m2461.write(':SOUR:SWE:SPAC LIN')
    m2461.write("TRAC:CLE")
    m2461.write('TRAC:POIN %f' %stepNum)
    m2461.write(':FORM:ELEM:SENS VOLT,CURR')
    m2461.write(':TRIG:COUN %f' % stepNum)
    m2461.write(':SOUR:SWE:POIN %f' %stepNum)
    m2461.write(':SOUR:DEL %0.2f' % delay)
    m2461.write(':OUTP ON')
    m2461.write(":INIT")
    m2461.write('*WAI')
    m2461.write(':OUTP OFF')

    time.sleep(1)

    data = m2461.query_ascii_values("FETC?")
    #print(data)
    # Read data from buffer of Keithley 2461 and write to the file"
    """
    lstCurrent = data[1::2] # get all odd elements, which are currents

    lstVoltage = data[0::2] # get all even elements, which are voltages

    # if Vstart > Vstop:
    #     Vstep = -abs(Vstep)
    # else:
    #     Vstep = abs(Vstep)
    lstAbsCurrent = np.abs(lstCurrent)
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(stepNum)]
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['iCompliance', iCompliance])
    #dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite = [['V', 'I', 'V_cal','abs_I']]
    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstCalVoltage[i] , lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")

    fig, ax = plt.subplots()
    line, = ax.plot(lstCalVoltage, lstAbsCurrent, 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A)')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')
    ax.set_yscale('log')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    
    
    
    

### Pico Ampmeter

In [ ]:
# picoamp meter
pico = rm.open_resource('GPIB0::22::INSTR')
pico.timeout = 10000
pico.write('*RST')

In [22]:
def fncMeasureOnePico(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='230530\\Dev02\\Test',\
                  askToStart=False):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    vRange = 500
    vILIM = 2e-3
    iRange = '2e-9'
    iNPLC = 1
    #vILIM = 'SOUR:VOLT:ILIM' + vILIM
    iRange = ':SENSE:CURR:RANG ' + iRange
    breakDownVolt = 505
    breakDownCurr = 2.5e-3
    pico.write('*RST')
    pico.write(iRange)
    pico.write("SOUR:VOLT:RANG %f" % vRange)
    pico.write("SOUR:VOLT:ILIM %f" %vILIM)
    pico.write(":SENSE:CURR:NPLC %f" %iNPLC )
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    data = [['Volt', 'Curr'],['V', 'A']]
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    
#     messStartMeasurement = 'File Paht: ' + filePath + '.\n' + " Do you want to start the measurement?" 
#     # Display a message box and ask for the operator's choice
#     choice = messagebox.askquestion("Measurement", messStartMeasurement)
    
#     if choice == 'yes':
    plt.show()
    time.sleep(1)
    pico.write(':SOUR:VOLT:STAT ON')
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
        lstVoltage.append(Vstart + i*Vstep)
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        pico.write(':INIT:IMM')
        #strVol = '{0:.2f}'.format(i*0.2)
        strVol = ':SOUR:VOLT {0:.2f}'.format(Vstart + i*Vstep)
        pico.write(strVol)
        results = pico.query(':MEASure:CURRent?')
        #print(results)
        current = results.split(',')[0][:-1]
        #voltage = l
        lstCurrent.append(float(np.squeeze(current)))
        lstAbsCurrent.append(np.abs(float(np.squeeze(current))))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs(float(np.squeeze(current))) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(float(np.squeeze(current))) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        
        
        time.sleep(0.1)
    pico.write(':SOUR:VOLT:STAT OFF')
    dataToWrite = [['vRange', vRange]]
    dataToWrite.append(['vILIMIT', vILIM])
    dataToWrite.append(['iRange', iRange[16:]])
    dataToWrite.append(['iNPLC', iNPLC])
    dataToWrite.append(['data Number', stepNum])
    dataToWrite.append(['V_cal', 'I','abs_I'])
    lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([round(Vstart + i * Vstep, 2), lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    # plt.savefig(os.path.splitext(filePath)[0] + '.png')
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')

In [ ]:
#run picoamp meter
arrPicoVMeasMulti =["pico", [[0, 1.5, 0.01],\
                [0, -1.5, -0.01],\
                [0, -500, -1],\
                 [0, 1.5, 0.01],\
                 [0, -1.5, -0.01]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath)

### Model 2657A

In [ ]:
#rm.close()
rm = pyvisa.ResourceManager()
m2657A = rm.open_resource('TCPIP0::132.181.53.204::inst0::INSTR')
m2657A.timeout = 10000
m2657A.write('*RST')
# print(rm.last_status)


StatusCode.error_resource_not_found


In [ ]:
def setM2657AIVMeasurement (iRange = 0.02, iLimit = 0.02, nplc = 1, delay = 0.05):
    # Reset and initialize instrutment
    m2657A.write(f"reset()")
    m2657A.write(f"status.reset()")
    m2657A.write(f"errorqueue.clear()")
    
    # Configure source function as 2W DCVOLTS
    m2657A.write(f"smua.source.func = smua.OUTPUT_DCVOLTS")
    m2657A.write(f"smua.sense = smua.SENSE_LOCAL")
    m2657A.write(f"display.screen = display.SMUA")
    m2657A.write(f"display.smua.measure.func = display.MEASURE_DCAMPS")
    m2657A.write(f"display.smua.digits = display.DIGITS_5_5")

    #Set up current compliance
    m2657A.write(f"smua.source.limiti = %f" %iLimit)

    #Set up current range 
    #Set up current range
    if iRange == "auto":
        m2657A.write(f"smua.measure.autorangei = smua.AUTORANGE_ON")
    else:
        m2657A.write(f"smua.measure.autorangei = smua.AUTORANGE_OFF")
        m2657A.write(f"smua.measure.rangei = %f" %iRange)

    

    #Set the measurement integration time
    m2657A.write(f"smua.measure.nplc = %f" %nplc)
    m2657A.write(f"smua.measure.delay = %f" %delay)

    

    


    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
    # m2657A.write(f"")
	
    m2657A.write(f"beeper.beep(0.5,100)")
    return True
def measureM2657AOnePoint(setV, outputON = False, resetV = True):
    #setM2657AIVMeasurement()
    m2657A.write(f"smua.source.levelv = %f" %setV)
    
    #Configure the reading buffers
    m2657A.write(f"smua.nvbuffer1.clear()")
    m2657A.write(f"smua.nvbuffer1.appendmode = 1")
    m2657A.write(f"smua.nvbuffer1.collecttimestamps = 1")
    m2657A.write(f"smua.nvbuffer1.collectsourcevalues = 0")
    m2657A.write(f"smua.nvbuffer1.fillmode = smua.FILL_ONCE")
    m2657A.write(f"smua.nvbuffer2.clear()")
    m2657A.write(f"smua.nvbuffer2.appendmode = 1")
    m2657A.write(f"smua.nvbuffer2.collecttimestamps = 1")
    m2657A.write(f"smua.nvbuffer2.collectsourcevalues = 0")
    m2657A.write(f"smua.nvbuffer2.fillmode = smua.FILL_ONCE")
    
    
    if (not outputON):
        m2657A.write(f"smua.source.output = 1")
    m2657A.write(f"smua.measure.i(smua.nvbuffer1)")
    m2657A.write(f"smua.measure.v(smua.nvbuffer2)")
    #time.sleep(2)

    #Turn off the output
    if resetV:
        m2657A.write(f"smua.source.levelv = 0")
        m2657A.write(f"smua.source.output = 0")
    
    m2657A.write(f"printbuffer(1, 1, smua.nvbuffer1.readings, smua.nvbuffer2.readings)")
    rawData = m2657A.read()
    # print("%g\t%g\t%g", smua.nvbuffer1.timestamps[1], smua.nvbuffer1.readings[1], smua.nvbuffer2.readings[1])
    arrRawData = rawData.split(",")
    #print ("rawData: ", rawData)

    return [float(arrRawData[i]) for i in range(len(arrRawData))]
setM2657AIVMeasurement(0.02, 0.02, 1, 0.05)
# rur = measureM2657AOnePoint(1,False,True)
# print (results)
current, voltage = measureM2657AOnePoint(1,False,True)
print ("Current: ", current, "Voltage: ", voltage)

In [ ]:
def fncMeasureOneM2657A(arrVMeasurement, \
                  Sample = 'Test', sweepNum = 1,\
                  folderPath='241108\\Dev02\\Test',\
                  askToStart=False,\
                  vRange = 200,\
                  iRange = "auto",\
                  iLimit = 2e-2,\
                  nplc = 1, \
                  delay = 0.05,\
                  breakDownVolt = 1500,\
                  breakDownCurr = 0.02):
    Vstart, Vstop, Vstep = arrVMeasurement
    fileName = "Sweep"+str(sweepNum)+Sample+'Star'+str(Vstart).replace("-","M")+'_Stop'+str(Vstop).replace("-","M")+'_Step'+str(Vstep).replace("-", "M")+'.csv'
    filePath = os.path.join(folderPath, fileName)
    print(filePath)
    stepNum = np.floor(abs((Vstart - Vstop)/Vstep)).astype(int) + 1
    # vRange = 500
    # vILIM = 2e-3
    # iRange = '2e-9'
    #iNPLC = 1
    #vILIM = 'SOUR:VOLT:ILIM' + vILIM
    #iRange = ':SENSE:CURR:RANG ' + iRange



    setM2657AIVMeasurement(iRange, iLimit, nplc, delay)
    absVstart = abs(Vstart)
    absVstop = abs(Vstop)
    # Set up source range
    if abs(Vstart) > abs(Vstop): 
        m2657A.write(f"smua.source.rangev = %f" %absVstart)
    else:
        m2657A.write(f"smua.source.rangev = %f" %absVstop)
    
    lstCurrent = []
    lstVoltage = []
    lstAbsCurrent = []
    
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'o-')
    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Absolute Current (A)')
    ax.set_yscale('log')
    
    

    plt.show()
    time.sleep(1)
    m2657A.write(f"smua.source.output = 1")
    curExceedLimitMeasNum = 0 # number of times the current exceeds the current limit - error
    afterBreakDownMeasNum = 10 # number of additional measurements after the current reaches the current limit at the flat region
    curLimitError = 0.0003
    curLimitLowerBound = breakDownCurr - curLimitError
    # Measurement loop
    for i in range(stepNum):
        
        if np.absolute(Vstart + i*Vstep) > breakDownVolt:
            break
        #pico.write(':INIT:IMM')
        #strVol = '{0:.2f}'.format(i*0.2)
        #strVol = ':SOUR:VOLT {0:.2f}'.format(Vstart + i*Vstep)
        #pico.write(strVol)
        current, voltage = measureM2657AOnePoint(Vstart + i*Vstep,True,False)
        
        # results = pico.query(':MEASure:CURRent?')
        # #print(results)

        # current = results.split(',')[0][:-1]
        #voltage = l
        lstVoltage.append(voltage)
        lstCurrent.append(current)
        lstAbsCurrent.append(abs(current))
        update_plot(fig, ax, line,lstVoltage, lstAbsCurrent)
        if (abs((current)) >  breakDownCurr): #or (stop_flag == True)
            break
        if (abs(current) >  curLimitLowerBound):
            curExceedLimitMeasNum+=1
        if (curExceedLimitMeasNum >= afterBreakDownMeasNum):
            break
        time.sleep(0.1)
    #Turn off the output
    m2657A.write(f"smua.source.levelv = 0")
    m2657A.write(f"smua.source.output = 0")

    
    dataToWrite = [['vRange: ', vRange]]
    dataToWrite.append(['iLimit: ', iLimit])
    dataToWrite.append(['iRange: ', iRange[16:]])
    dataToWrite.append(['nplc: ', nplc, 'delay: ', delay])
    dataToWrite.append(['data Number:', stepNum])
    dataToWrite.append(['V_meas', 'I','abs_I'])
    # lstCalVoltage = [(Vstart + i*Vstep) for i in range(len(lstAbsCurrent))]

    for i in range(len(lstCurrent)):
        dataToWrite.append([lstVoltage[i], lstCurrent[i], lstAbsCurrent[i]])
    try:
        with open(filePath, 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerows(dataToWrite)
            file.close()
    except IOError:
        print("Error: Could not open file for writing")
    plt.show()
    # plt.savefig(os.path.splitext(filePath)[0] + '.png')
    plt.savefig(os.path.splitext(filePath)[0] + '_log.png')
    ax.set_yscale('linear')
    plt.show()
    plt.savefig(os.path.splitext(filePath)[0] + '_lin.png')

In [ ]:
m2657A.write(f"beeper.beep(2,100)")

In [ ]:
sweepFilePath = "C:\\Users\\gtd19\\Semsem\\bGa2O3\\m2461leyControl\\TSP_2657A\\2657A_DiodeRL\\DiodeRL.tsp"

m2657A.write("loadscript simpleSweep")
with open(sweepFilePath) as fp:
    for line in fp:
        m2657A.write(line)
m2657A.write("endscript")
m2657A.write("simpleSweep.run()")
fp.close()

In [ ]:
#run M2657A
arrM2657AVMeasMulti =["M2657A", [[0,-2,-0.01],[0,3,0.01]]]
fncMeasureMultiple(arrM2657AVMeasMulti,sample, 1, folderPath)

In [ ]:
#run M2657A
arrM2657AVMeasMulti =["M2657A", [[0,-1.5,-0.01],[0,1.5,0.01],[0,1500,1],[0,-1.5,-0.01],[0,1.5,0.01]]]
fncMeasureMultiple(arrM2657AVMeasMulti,sample, 3, folderPath)

In [ ]:
m2657A.write(f"smua.source.levelv = 0")
m2657A.write(f"smua.source.output = 0")

In [5]:
rm.close()

In [ ]:
# Reset Model 2657A
rm = pyvisa.ResourceManager()
m2657A = rm.open_resource('TCPIP0::132.181.53.204::inst0::INSTR')
m2657A.timeout = 10000
m2657A.write('*RST')

In [21]:
#run M2657A
arrM2657AVMeasMulti =["M2657A", [[0,3000,100]]]
fncMeasureMultiple(arrM2657AVMeasMulti,sample, 3, folderPath)

241114_Fab231219\DTest\Air\Sweep3AirStar0_Stop3000_Step100.csv


### Model 2461

### For references

In [ ]:
#run picoamp meter
arrPicoVMeasMulti =["M2657A", [[0, -1.5, -0.1],\
                [0, 1.5, 0.1],\
                [0, 500, 1],\
                 [0, -1.5, -0.1],\
                 [0, 1.5, 0.1]]]
fncMeasureMultiple(arrPicoVMeasMulti,sample, 3, folderPath)

In [ ]:
arrVMeas = [0, -3, -0.01]
fncMeasureOne(arrVMeas, sample, 1, folderPath)

In [ ]:
#run Keithley 2400 [0, 3, 0.01],\
arrKLVMeasMulti =["keith1",[[0, 3, 0.01],\
                            [0, 3, 0.01],\
                            [0, -3, -0.01]]]
fncMeasureMultiple(arrKLVMeasMulti,sample, 0, folderPath)